# Week 9 과제 1

**잎 사진을 통한 질병 분류**

*   데이터: 21개의 클래스로 구분된 약 20000장의 이미지
><img src="https://drive.google.com/uc?id=1YQkxnNy61Gyi3Gp6ylCKeS72BVruJXr_" width="700" height="500"> 
*   데이터 전처리
    *   전체 데이터를 train,validation,test로 분할해주세요
    *   저는 6:2:2로 분할했는데, 비율은 바꾸셔도 무방합니다.
   
    
*   학습 진행 방향
    *   Baseline 모델(pre-trained model 사용X) 구축
    *   Pre-trained 모델 사용
    *   Baseline과 ResNet50 모델의 성능을 비교해주세요 ~!

In [2]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')
# 경로 설정
import os
os.chdir('/content/gdrive/My Drive/')

Mounted at /content/gdrive


## 0. 데이터 분할

* 데이터 분할을 위한 디렉토리 생성

In [3]:
import os
import shutil
DIR = '/content/gdrive/MyDrive/Colab Notebooks/tobigs15/'
original_dataset_dir = DIR+'tobigs_week9_plant_leaf/'   
classes_list = os.listdir(original_dataset_dir) 
 
base_dir = DIR+'tobigs_week9_splitted/' 

In [ ]:
os.mkdir(base_dir)
 
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

* 데이터 분할과 클래스별 데이터 수 확인

In [ ]:
import math
 
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)


Train size( Potato___Early_blight ):  600
Validation size( Potato___Early_blight ):  200
Test size( Potato___Early_blight ):  200
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Grape___healthy ):  253
Validation size( Grape___healthy ):  84
Test size( Grape___healthy ):  84
Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Potato___Late_blight ):  600
Validation size( Potato___Late_blight ):  200
Test size( Potato___Late_blight ):  200
Train size( Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ):  645
Validation size( Grape___Leaf_blight_(Isariopsis_Leaf_

## 1. 베이스라인 모델을 구축해 주세요
* Pre-Trained Model을 사용하지 않고 직접 모델을 구축해 주세요 !


In [4]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

In [5]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
train_dataset = ImageFolder(root=base_dir+'train/', transform=transform_base) 
val_dataset = ImageFolder(root=base_dir+'val/', transform=transform_base)

In [6]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* 베이스라인 모델 설계하기

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module): 
  
    def __init__(self): 
    
        super(Net, self).__init__()
        conv1 = nn.Conv2d(in_channels=3, out_channels = 64, kernel_size = (5,5), stride = 1, padding = 2) # (64+2*2-5)/1 + 1 = 64
        relu = nn.ReLU()
        maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        conv2 = nn.Conv2d(in_channels=64, out_channels = 64, kernel_size = (3,3), stride = 1, padding = 2)  
        conv3 = nn.Conv2d(in_channels=64, out_channels = 32, kernel_size = (3,3), stride = 2, padding = 1)     
        # conv4 = nn.Conv2d(in_channels=256, out_channels = 128, kernel_size = (3,3), stride = 1)   
        fc1 = nn.Linear(32*4*4, 128)
        fc2 = nn.Linear(128, 21)
        
        self.convnet = nn.Sequential(
            conv1, relu, maxpool2, # 32
            conv2, relu, maxpool2, # 17
            conv3, relu, maxpool2 # 4
        )
        self.fc = nn.Sequential(
            fc1, relu,
            fc2
        )
    
    def forward(self, x):  

        output = self.convnet(x)
        output = output.view(x.size(0), -1)
        output = self.fc(output)

        return F.log_softmax(output, dim=1)  

model_base = Net().to(DEVICE)  
optimizer = optim.Adam(model_base.parameters(), lr=0.005) 

* 모델 학습을 위한 함수
* 모델 학습, 평가를 위한 가이드라인 코드입니다. 꼭 이 코드를 사용하지는 않으셔도 됩니다 !!

In [8]:
def train(model, train_loader, optimizer, criterion, scheduler):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) 
        optimizer.zero_grad()
        output = model(data) 
        loss = criterion(output, target)      #Cross Entropy Loss 사용했습니다
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

* 모델 평가를 위한 함수

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() 
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

* 모델 학습을 실행하기

In [10]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30, criterion = nn.CrossEntropyLoss(), scheduler = None):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer, criterion, scheduler)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, train_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

    model.load_state_dict(best_model_wts)  
    return model
	

In [ ]:
base = train_baseline(model_base, train_loader, val_loader, optimizer)  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-------------- epoch 1 ----------------
train Loss: 2.1222, Accuracy: 32.34%
val Loss: 2.1222, Accuracy: 32.34%
Completed in 1m 6s
-------------- epoch 2 ----------------
train Loss: 1.3907, Accuracy: 53.70%
val Loss: 1.3907, Accuracy: 53.70%
Completed in 1m 6s
-------------- epoch 3 ----------------
train Loss: 0.9552, Accuracy: 67.73%
val Loss: 0.9552, Accuracy: 67.73%
Completed in 1m 7s
-------------- epoch 4 ----------------
train Loss: 0.7298, Accuracy: 75.94%
val Loss: 0.7298, Accuracy: 75.94%
Completed in 1m 7s
-------------- epoch 5 ----------------
train Loss: 0.5816, Accuracy: 80.13%
val Loss: 0.5816, Accuracy: 80.13%
Completed in 1m 7s
-------------- epoch 6 ----------------
train Loss: 0.5221, Accuracy: 81.69%
val Loss: 0.5221, Accuracy: 81.69%
Completed in 1m 7s
-------------- epoch 7 ----------------
train Loss: 0.4782, Accuracy: 83.30%
val Loss: 0.4782, Accuracy: 83.30%
Completed in 1m 7s
-------------- epoch 8 ----------------
train Loss: 0.3264, Accuracy: 89.20%
val Lo

In [ ]:
torch.save(base,base_dir + 'baseline.pt')

## 2. Transfer Learning 모델 학습

* Transfer Learning을 위한 준비
* Transfer Learning이 익숙하지 않으신 분들은 PyTorch에서 제공하는 https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/transfer_learning_tutorial.html 을 참고하세요 :)

In [26]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([64,64]), 
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        # transforms.CenterCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    
    'val': transforms.Compose([transforms.Resize([64,64]),  
        'validation 데이터는 Test 데이터와 동일한 조건이어야 합니다.'
        'validation에 맞는 전처리를 수행해 주세요',
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        # transforms.CenterCrop(52),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

### Pre-Trained Model 불러오기
- 저는 ResNet50을 사용했는데, 코랩 기준으로 다른 ResNet계열이나 DenseNet 정도까지는 큰 무리 없이 훈련할 수 있습니다. Unfreeze layer 수가 적으면 다른 모델도 사용할 수 있을 것입니다.
- 한 가지 모델을 선택해서 Transfer Learning을 해 주세요 !!

In [43]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, 21)
resnet = resnet.to(DEVICE)
 
criterion = nn.CrossEntropyLoss() 
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.01)
 
from torch.optim import lr_scheduler  #scheduler는 사용하지 않으셔도 됩니다 (선택사항)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)  #7에폭마다 learning rate를 조절하는 역할을 합니다

* Pre-Trained Model의 일부 Layer Freeze하기 (resnet 기준입니다 !!)

In [44]:
ct = 0 
for child in resnet.children():  
    ct+= 1  
    print("#{}".format(ct), child)
    if ct < 6: 
        for param in child.parameters():
            param.requires_grad = False

#1 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#2 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#3 ReLU(inplace=True)
#4 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
#5 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1,

* Fine Tuning을 진행해주세요 ~!

In [45]:
train_dataset = ImageFolder(root=base_dir+'train/', transform=data_transforms['train']) 
val_dataset = ImageFolder(root=base_dir+'val/', transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [46]:
resnet_trained = train_baseline(resnet, train_loader, val_loader, optimizer_ft, criterion = nn.CrossEntropyLoss(), num_epochs=15)

-------------- epoch 1 ----------------
train Loss: 3.0335, Accuracy: 34.75%
val Loss: 2.9867, Accuracy: 34.39%
Completed in 1m 57s
-------------- epoch 2 ----------------
train Loss: 0.7827, Accuracy: 75.98%
val Loss: 0.7883, Accuracy: 75.64%
Completed in 1m 57s
-------------- epoch 3 ----------------
train Loss: 0.5041, Accuracy: 82.86%
val Loss: 0.6633, Accuracy: 82.81%
Completed in 1m 57s
-------------- epoch 4 ----------------
train Loss: 0.2453, Accuracy: 90.99%
val Loss: 0.2463, Accuracy: 91.11%
Completed in 1m 57s
-------------- epoch 5 ----------------
train Loss: 0.2795, Accuracy: 91.05%
val Loss: 0.2853, Accuracy: 90.47%
Completed in 1m 57s
-------------- epoch 6 ----------------
train Loss: 0.2462, Accuracy: 91.71%
val Loss: 0.2385, Accuracy: 92.14%
Completed in 1m 57s
-------------- epoch 7 ----------------
train Loss: 0.0803, Accuracy: 97.02%
val Loss: 0.0767, Accuracy: 97.23%
Completed in 1m 57s
-------------- epoch 8 ----------------
train Loss: 0.0891, Accuracy: 96.97%

In [47]:
torch.save(resnet_trained, base_dir + 'resnet50_.pt')

## 모델 평가

* 모델 평가를 위해서는 평가 데이터 또한 전처리를 해주어야 합니다.
* validation 데이터와 동일하게 전처리를 해 주세요 ~!!

* 베이스라인 모델 평가를 위한 전처리하기

In [ ]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
test_base = ImageFolder(root=base_dir+'test/',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* Transfer Learning모델 평가를 위한 전처리하기

In [49]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        # "채워주세요"
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root=base_dir+'test/', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### 성능 평가하기
* 저는 여기서 accuracy만을 평가했지만, 분류 모델이기에 다양한 방법의 평가가 가능합니다.
* Confusion Matrix를 이용한 비교도 가능하고, 한 작물에 해당하는 클래스가 여러개인 다중 분류에서 F1-score를 계산하는것도 의미가 있을 것입니다. 
* 다양한 시도를 하시는 분께 가산점 드리겠습니다 :):)

* 베이스라인 모델 성능 평가하기

In [ ]:
baseline=torch.load(base_dir+'baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   91.86652763295099


* Transfer Learning 모델 성능 평가하기

In [50]:
resnet50=torch.load(base_dir+'resnet50_.pt') 
# resnet50 = resnet_trained
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   97.21063607924921


* 두 모델의 성능을 비교 평가하는 설명을 작성해주세요 ~!!

base로는 CNN layer 3개와 FC layer 2개로 이뤄진 모델을 구상하였다. 그 결과, test accuracy는 91.87%로 나왔다. 
Transfer Learning은 ResNet 50모델에서 앞에서 5개의 레이어를 freeze한 후 fine tuning을 진행하였다. 그 결과, base model보다 훨씬 빠르게 학습이 진행되었으며 최종적으로 test accuracy는 97.2%로 나왔다.